In [1]:
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Conv2D, Pool2D, Linear

# 定义模型结构
class MNIST(fluid.dygraph.Layer):
     def __init__(self):
         super(MNIST, self).__init__()

         # 定义一个卷积层，使用relu激活函数
         self.conv1 = Conv2D(num_channels=1, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义一个池化层，池化核为2，步长为2，使用最大池化方式
         self.pool1 = Pool2D(pool_size=2, pool_stride=2, pool_type='max')
         # 定义一个卷积层，使用relu激活函数
         self.conv2 = Conv2D(num_channels=20, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义一个池化层，池化核为2，步长为2，使用最大池化方式
         self.pool2 = Pool2D(pool_size=2, pool_stride=2, pool_type='max')
         # 定义一个全连接层，输出节点数为10
         self.fc = Linear(input_dim=980, output_dim=10, act='softmax')
    # 定义网络的前向计算过程
     def forward(self, inputs, label):
         x = self.conv1(inputs)
         x = self.pool1(x)
         x = self.conv2(x)
         x = self.pool2(x)
         x = fluid.layers.reshape(x, [x.shape[0], 980])
         x = self.fc(x)
         if label is not None:
             acc = fluid.layers.accuracy(input=x, label=label)
             return x, acc
         else:
             return x

In [2]:
from pathlib import Path
import numpy as np
#optimizer中指定更新参数阻止反向传播
with fluid.dygraph.guard(fluid.CUDAPlace(0)):
    model = MNIST() #如果加载参数已经训练稳定，fc.w可能不会变。如果加载模型训练次数少，fc.w还会变。
    model.train()
    params=model.parameters()
    print("param name:{},value:{}".format(params[0].name,params[0].numpy()[0][0][0][:5]))
    update_params=[]
    for param in params:
        #print(param.__class__)
        param_name=param.name
        if 'fc' in param_name or 'linear' in param_name:
            update_params.append(param)
    print(list(map(lambda x:x.name,update_params)))

    best_model_path='mnist'
    if Path(best_model_path+".pdparams").exists():
        print('试着加载模型文件.....')
        pretrained_model, _ = fluid.load_dygraph(best_model_path)
        model.load_dict(pretrained_model)
        print("模型初始化完成")
    print("param name:{},value:{}".format(params[0].name,params[0].numpy()[0][0][0][:5]))
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01, parameter_list=update_params)

    batch_size=64
    trainset = paddle.dataset.mnist.train()
    # 包装数据读取器，每次读取的数据数量设置为batch_size=8
    train_reader = paddle.batch(paddle.reader.shuffle(trainset,buf_size=10*batch_size), batch_size=batch_size)
    conv_w=[]
    fc_w=[]#训练稳定可能不变
    fc_b=[]
    EPOCH_NUM = 2
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_reader()):

            #准备数据
            img_data = np.array([x[0] for x in data]).astype('float32').reshape(-1,1,28,28)
            label_data = np.array([x[1] for x in data]).astype('int64').reshape(-1,1)

            image = fluid.dygraph.to_variable(img_data)
            label = fluid.dygraph.to_variable(label_data)

            #前向计算的过程，同时拿到模型输出值和分类准确率
            predict, acc = model(image, label)

            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)

            #每训练了200批次的数据，打印下当前Loss的情况
            if batch_id%500==0:
                conv_w.append("param name:{},value:{}".format(params[0].name,params[0].numpy()[0][0][0][:5]))
                fc_w.append("param name:{},value:{}".format(params[-2].name,params[-2].numpy()[0][-5:]))
                fc_b.append("param name:{},value:{}".format(params[-1].name,params[-1].numpy()[:5]))
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))

            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

    def print_self(vals):
        for val in vals:print(val)

    print("不变-----")
    print(print_self(conv_w))
    print("变-----")
    print(print_self(fc_w))
    print("变-----")
    print(print_self(fc_b))

param name:conv2d_0.w_0,value:[ 0.16167769 -0.3103957   0.50627756 -0.7470018  -0.01521146]
['linear_0.w_0', 'linear_0.b_0']
试着加载模型文件.....
模型初始化完成
param name:conv2d_0.w_0,value:[-0.11276215 -0.02700309  0.09193268  0.10492735 -0.0250226 ]
epoch: 0, batch: 0, loss is: [0.43705887], acc is [0.875]
epoch: 0, batch: 200, loss is: [0.41092676], acc is [0.90625]
epoch: 0, batch: 400, loss is: [0.37229502], acc is [0.921875]
epoch: 0, batch: 600, loss is: [0.3291864], acc is [0.875]
epoch: 0, batch: 800, loss is: [0.32874358], acc is [0.90625]
epoch: 1, batch: 0, loss is: [0.33340156], acc is [0.9375]
epoch: 1, batch: 200, loss is: [0.52279055], acc is [0.78125]
epoch: 1, batch: 400, loss is: [0.5991076], acc is [0.875]
epoch: 1, batch: 600, loss is: [0.28933373], acc is [0.9375]
epoch: 1, batch: 800, loss is: [0.6618682], acc is [0.84375]
不变-----
param name:conv2d_0.w_0,value:[-0.11276215 -0.02700309  0.09193268  0.10492735 -0.0250226 ]
param name:conv2d_0.w_0,value:[-0.11276215 -0.02700309 